In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import re
train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")
train1 = train.drop(['Descript', 'Resolution'], axis = 1)

In [24]:
train1 = train1.query('Y != 90')

In [25]:
# Case9 PdDistrictのみone-hot train1.ver

Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(['Dates'], axis = 1, inplace = True)
def to_season(month_num):
    season = 'winter'
    if 3<=month_num<=5:
        season = 'spring'
    elif 6<=month_num<=8:
        season = 'summer'
    elif 9<=month_num<=11:
        season = 'autumn'
    return season
train1['Season'] =Dt.dt.month.apply(to_season)

le = LabelEncoder()
train1['DayOfWeek'] = le.fit_transform(train1['DayOfWeek'])

Ad_dict = {'.* / .*':0, '.*':1}
train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)

train1['Category'] = le.fit_transform(train1['Category'])

train1_PD = pd.get_dummies(train1['PdDistrict'])

train1 = pd.concat([train1, train1_PD], axis = 1)

train1.drop(['PdDistrict'], axis = 1, inplace=True)

train1.head()

,Category,DayOfWeek,Address,X,Y,Date,Year,Month,Day,Hour,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,37,6,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,...,0,0,0,0,1,0,0,0,0,0
1,21,6,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,...,0,0,0,0,1,0,0,0,0,0
2,21,6,0,-122.424363,37.800414,2015-05-13,2015,5,13,23,...,0,0,0,0,1,0,0,0,0,0
3,16,6,1,-122.426995,37.800873,2015-05-13,2015,5,13,23,...,0,0,0,0,1,0,0,0,0,0
4,16,6,1,-122.438738,37.771541,2015-05-13,2015,5,13,23,...,0,0,0,0,0,1,0,0,0,0


In [26]:
train1_SS = pd.get_dummies(train1['Season'])

train1 = pd.concat([train1, train1_SS], axis = 1)

train1.drop(['Season'], axis = 1, inplace=True)

In [27]:
# Case9 PdDistrictのみone-hot test.ver

Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
test.drop(['Dates'], axis = 1, inplace = True)
def to_season(month_num):
    season = 'winter'
    if 3<=month_num<=5:
        season = 'spring'
    elif 6<=month_num<=8:
        season = 'summer'
    elif 9<=month_num<=11:
        season = 'autumn'
    return season
test['Season'] =Dt.dt.month.apply(to_season)

le = LabelEncoder()
test['DayOfWeek'] = le.fit_transform(test['DayOfWeek'])

Ad_dict = {'.* / .*':0, '.*':1}
test['Address'] = test['Address'].replace(Ad_dict,regex = True)


test_PD = pd.get_dummies(test['PdDistrict'])

test = pd.concat([test, test_PD], axis = 1)

test.drop(['PdDistrict'], axis = 1, inplace=True)

test.head()

,Id,DayOfWeek,Address,X,Y,Date,Year,Month,Day,Hour,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,3,1,-122.399588,37.735051,2015-05-10,2015,5,10,23,...,1,0,0,0,0,0,0,0,0,0
1,1,3,0,-122.391523,37.732432,2015-05-10,2015,5,10,23,...,1,0,0,0,0,0,0,0,0,0
2,2,3,1,-122.426002,37.792212,2015-05-10,2015,5,10,23,...,0,0,0,0,1,0,0,0,0,0
3,3,3,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,...,0,0,1,0,0,0,0,0,0,0
4,4,3,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,...,0,0,1,0,0,0,0,0,0,0


In [28]:
test_SS = pd.get_dummies(test['Season'])

test = pd.concat([test, test_SS], axis = 1)

test.drop(['Season'], axis = 1, inplace=True)

In [29]:
cols = ['DayOfWeek', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour',"spring","summer","autumn","winter", 'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN']
X = train1[cols]
Y = train1['Category']
X_test = test[cols]

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [30]:
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

In [31]:
accuracy_score(y_val, y_pred)

0.2903851629681657

In [32]:
xgb_submission = model.predict(X_test)

In [34]:
import csv
pred = model.predict_proba(test[['DayOfWeek', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour',"spring","summer","autumn","winter", 'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)
submission.head()

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.003046,0.244904,0.000059,0.000990,0.027776,0.000721,0.004493,0.011991,0.002784,...,0.000122,0.014359,0.000230,0.029479,9.583200e-07,0.006737,0.089103,0.127484,0.031845,0.030032
1,1,0.001794,0.059702,0.000019,0.000587,0.000818,0.000428,0.012324,0.037201,0.005232,...,0.000062,0.003647,0.001067,0.022183,1.628000e-05,0.000211,0.018710,0.086822,0.091896,0.020054
2,2,0.001827,0.045968,0.000028,0.000035,0.140971,0.000917,0.001159,0.011713,0.002412,...,0.000004,0.005799,0.000092,0.021316,7.842236e-07,0.013604,0.063937,0.112233,0.010344,0.003517
3,3,0.001532,0.101603,0.000013,0.001431,0.027286,0.001448,0.002933,0.012934,0.006172,...,0.000155,0.006182,0.000106,0.037232,7.669078e-07,0.006100,0.078903,0.143228,0.031251,0.041447
4,4,0.001532,0.101603,0.000013,0.001431,0.027286,0.001448,0.002933,0.012934,0.006172,...,0.000155,0.006182,0.000106,0.037232,7.669078e-07,0.006100,0.078903,0.143228,0.031251,0.041447


In [35]:
submission.to_csv('XGB_9.csv',index = False)